# Rupaul's Drag Race Machine

In [74]:
import pandas as pd
from scipy.stats import rankdata, kendalltau

theData = pd.read_csv("dragrace.csv")
theData= theData.fillna(0)
#theData.iloc[1:10,:]

In [75]:
queens = theData.groupby('Name').max()
queens = queens.reset_index(drop=False)
#queens.iloc[1:10,:]

In [76]:
#queens.loc[queens.Place==1,:]

In [77]:
Xtrain = queens.loc[queens.Season<7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
Xtest = queens.loc[queens.Season==7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
ytrain = queens.loc[queens.Season<7,'Place']
ytest = queens.loc[queens.Season==7,'Place']
season8X = queens.loc[queens.Season==8,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
season8y = queens.loc[queens.Season==8,'Place']
season7 = queens.loc[queens.Season==7,['Name','Place']]
season7.columns = ['Name','Actual']
season7 = season7.reset_index(drop=True)
season8 = queens.loc[queens.Season==8,['Name','Place']]
season8.columns = ['Name','Actual']
season8 = season8.reset_index(drop=True)

# Support Vector Machine Classifier

In [78]:
from sklearn.svm import SVC
model = SVC()
model.fit(Xtrain,ytrain)

yfit = model.predict(Xtest)
yfitpd = pd.DataFrame(yfit)
yfitpd['Name'] = season7.Name
yfitpd.columns = ['Predicted','Name']
yfitpd = yfitpd.loc[:,['Name','Predicted']]
yfitpd['Predicted'] = rankdata(yfitpd.Predicted,method='min')
svc7 = pd.merge(season7,yfitpd).sort_values('Actual')
svc7

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,5
9,Pearl,3,9
5,Kennedy Davenport,4,5
4,Katya,5,5
12,Trixie Mattel,6,9
7,Miss Fame,7,3
1,Jaidynn Diore Fierce,8,12
6,Max,9,9
3,Kandy Ho,10,4


In [79]:
kendalltau(svc7.Actual,svc7.Predicted)

KendalltauResult(correlation=0.22130457664129397, pvalue=0.27024792506861006)

## Season 8 Predictions

In [80]:
presentFit = model.predict(season8X)
presentFitPD = pd.DataFrame(presentFit)
presentFitPD['Name'] = season8['Name']
presentFitPD.columns = ['Predicted','Name']
presentFitPD['Predicted'] = rankdata(presentFitPD.Predicted,method='min')
svc8 = pd.merge(season8,presentFitPD).sort_values('Predicted')
svc8

,Name,Actual,Predicted
0,Acid Betty,8,1
8,Naomi Smalls,0,2
1,Bob the Drag Queen,0,3
4,Dax ExclamationPoint,11,3
5,Derrick Barry,0,3
9,Naysha Lopez,9,3
10,Robbie Turner,0,3
2,Chi Chi DeVayne,0,8
3,Cynthia Lee Fontaine,10,8
11,Thorgy Thor,0,8


# Gaussian Naive Bayes

In [81]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(Xtrain,ytrain)
nbTest = model.predict(Xtest)
nbTestDB = pd.DataFrame(nbTest)
nbTestDB['Name'] = season7.Name
nbTestDB.columns = ['Predicted','Name']
nbTestDB = nbTestDB.loc[:,['Name','Predicted']]
nbTestDB['Predicted'] = rankdata(nbTestDB.Predicted,method='min')

nb7 = pd.merge(season7,nbTestDB).sort_values('Actual')
nb7

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,6
9,Pearl,3,8
5,Kennedy Davenport,4,1
4,Katya,5,1
12,Trixie Mattel,6,8
7,Miss Fame,7,4
1,Jaidynn Diore Fierce,8,6
6,Max,9,8
3,Kandy Ho,10,4


In [82]:
kendalltau(nb7.Actual,nb7.Predicted)

KendalltauResult(correlation=0.48108780653777095, pvalue=0.016544547824374332)

## Season 8 Predictions

In [83]:
nb8 = model.predict(season8X)
nb8DB = pd.DataFrame(nb8)
nb8DB['Name'] = season8.Name
nb8DB.columns = ['Predicted','Name']
nb8DB['Predicted'] = rankdata(nb8DB.Predicted,method='min')
nb8 = pd.merge(season8,nb8DB).sort_values('Predicted')
nb8

,Name,Actual,Predicted
0,Acid Betty,8,1
1,Bob the Drag Queen,0,1
6,Kim Chi,0,1
11,Thorgy Thor,0,1
5,Derrick Barry,0,5
3,Cynthia Lee Fontaine,10,6
9,Naysha Lopez,9,6
2,Chi Chi DeVayne,0,8
8,Naomi Smalls,0,8
10,Robbie Turner,0,8


# Random Forest Classifier

In [84]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(Xtrain,ytrain)
yfit = model.predict(Xtest)

yfitpd = pd.DataFrame(yfit)
yfitpd['Name'] = season7.Name
yfitpd.columns = ['Predicted','Name']
yfitpd = yfitpd.loc[:,['Name','Predicted']]
yfitpd['Predicted'] = rankdata(yfitpd.Predicted,method='min')

## Season 7 Predictions

In [85]:
rfClass = pd.merge(season7,yfitpd).sort_values('Actual')
rfClass

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,6
9,Pearl,3,3
5,Kennedy Davenport,4,4
4,Katya,5,4
12,Trixie Mattel,6,6
7,Miss Fame,7,10
1,Jaidynn Diore Fierce,8,6
6,Max,9,1
3,Kandy Ho,10,6


In [86]:
kendalltau(rfClass.Actual,rfClass.Predicted)

KendalltauResult(correlation=0.64061851133006142, pvalue=0.0014157354382150771)

## Season 8 Predictions

In [87]:
rf8Fit = model.predict(season8X)
rf8FitPD = pd.DataFrame(rf8Fit)
rf8FitPD['Name'] = season8.Name
rf8FitPD.columns = ['Predicted','Name']
rf8FitPD = rf8FitPD.loc[:,['Name','Predicted']]
rf8FitPD['Predicted'] = rankdata(rf8FitPD.Predicted,method='min')

In [88]:
rfClass8 = pd.merge(season8,rf8FitPD).sort_values('Predicted')
rfClass8

,Name,Actual,Predicted
0,Acid Betty,8,1
1,Bob the Drag Queen,0,1
6,Kim Chi,0,3
10,Robbie Turner,0,4
2,Chi Chi DeVayne,0,5
5,Derrick Barry,0,6
11,Thorgy Thor,0,6
8,Naomi Smalls,0,8
9,Naysha Lopez,9,8
4,Dax ExclamationPoint,11,10


# Random Forest Regressor

In [89]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(200,random_state=24601)
forest.fit(Xtrain,ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=False, random_state=24601,
           verbose=0, warm_start=False)

## Season 7 Predictions

In [90]:
forestFit = forest.predict(Xtest)
forestFitDB = pd.DataFrame(forestFit.round())
forestFitDB['Name'] = season7.Name
forestFitDB.columns = ['Predicted','Name']
forestFitDB['Predicted'] = rankdata(forestFitDB.Predicted,method='min')
rfReg7 = pd.merge(season7,forestFitDB).sort_values('Actual')
rfReg7

,Name,Actual,Predicted
13,Voilet Chachki,1,2
0,Ginger Minj,2,1
9,Pearl,3,6
5,Kennedy Davenport,4,2
4,Katya,5,2
12,Trixie Mattel,6,8
7,Miss Fame,7,8
1,Jaidynn Diore Fierce,8,6
6,Max,9,5
3,Kandy Ho,10,8


In [91]:
kendalltau(rfReg7.Actual,rfReg7.Predicted)

KendalltauResult(correlation=0.73379938570534309, pvalue=0.00025655048033250812)

## Season 8 Predictions

In [92]:
forest8 = forest.predict(season8X)
forest8DB = pd.DataFrame(forest8.round())
forest8DB['Name'] = season8.Name
forest8DB.columns = ['Predicted','Name']
forest8DB['Predicted'] = rankdata(forest8DB.Predicted,method='min')
pd.merge(season8,forest8DB).sort_values('Predicted')

,Name,Actual,Predicted
1,Bob the Drag Queen,0,1
6,Kim Chi,0,2
2,Chi Chi DeVayne,0,3
5,Derrick Barry,0,3
10,Robbie Turner,0,3
11,Thorgy Thor,0,3
0,Acid Betty,8,7
8,Naomi Smalls,0,8
7,Laila McQueen,11,9
4,Dax ExclamationPoint,11,10


# Neural Network

Unfortunately, scikit learn's stable release does not include neural networks (it is being added in the next version) so if I want to include a neural network I have to code one from scratch (credit goes to Joel Grus's Data Science From Scratch book, available from O'Reilly, for much of the code for the neural network)

In [93]:
import numpy as np
import math, random
# create the "step" function
def sigmoid(t): 
    return 1 / (1 + math.exp(-t))

# create the neuron
def neuron_output(weights, inputs):   #This is a simpler representation; weights for input plus one extra (bias)
    return sigmoid(np.dot(weights, inputs))

# define the network
def feed_forward(neural_network, input_vector):
    """takes in a neural network (represented as a list of lists of lists of weights)
    and returns the output from forward-propagating the input"""

    outputs = []

    for layer in neural_network: #Remember the neural network is given as a list of "layers" which have neurons in them

        input_with_bias = input_vector + [1]          # add a bias input (this just allos us to use a dot product)
        output = [neuron_output(neuron, input_with_bias) # compute the output
                  for neuron in layer]                   # for this layer
        outputs.append(output)                           # and remember it

        # the input to the next layer is the output of this one
        input_vector = output

    return outputs

# define the back-propagation that allows the network to learn
def backpropagate(network, input_vector, target):

    hidden_outputs, outputs = feed_forward(network, input_vector)
    
    # the output * (1 - output) is from the derivative of sigmoid
    output_deltas = [output * (1 - output) * (output - target[i])
                     for i, output in enumerate(outputs)]
                     
    # adjust weights for output layer (network[-1])
    for i, output_neuron in enumerate(network[-1]):
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            output_neuron[j] -= output_deltas[i] * hidden_output

    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1 - hidden_output) * 
                      np.dot(output_deltas, [n[i] for n in network[-1]]) 
                     for i, hidden_output in enumerate(hidden_outputs)]

    # adjust weights for hidden layer (network[0])
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input

In [94]:
random.seed(0)   # to get repeatable results
input_size = 8  # each input is a vector of length 8
num_hidden = 5   # we'll have 5 neurons in the hidden layer
output_size = 14 # we need 14 outputs for each input, since there are 14 possible places

In [95]:
# each hidden neuron has one weight per input, plus a bias weight
hidden_layer = [[random.random() for __ in range(input_size + 1)]
                    for __ in range(num_hidden)]

In [96]:
# each output neuron has one weight per hidden neuron, plus a bias weight
output_layer = [[random.random() for __ in range(num_hidden + 1)]
                    for __ in range(output_size)]

In [97]:
# the network starts out with random weights
network = [hidden_layer, output_layer]

Neural networks work best if values are standardized close to 0-1. So for the non-dummy variables, I'll standardize to mean=0, sd=1

In [98]:
from sklearn.preprocessing import scale
queensNN = queens
queensNN['Age'] = scale(queensNN['Age'])
queensNN['Wins'] = scale(queensNN['Wins'])
queensNN['Highs'] = scale(queensNN['Highs'])
queensNN['Lows'] = scale(queensNN['Lows'])
queensNN['Season'] = scale(queensNN['Season'])
queensNN['Lipsyncs'] = scale(queensNN['Lipsyncs'])

#dataset = zip(inputs_dig, targets_dig)

/Users/thomas/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/Users/thomas/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [99]:
XtrainNN = queensNN.loc[queens.Season<7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
XtestNN = queensNN.loc[(queensNN.Season>1) & (queensNN.Season<1.5),['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
ytrainNN = queensNN.loc[queens.Season<7,'Place']
ytestNN = queensNN.loc[queens.Season==7,'Place']
season8XNN = queensNN.loc[queensNN.Season>1.5,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
season8yNN = queensNN.loc[queens.Season==8,'Place']

In [100]:
yNN = [[1 if i == j else 0 for i in range(14)]
               for j in ytrainNN ]
XNN = XtrainNN.values.tolist()

In [101]:
for i in range(10000):
    for input_vector, target_vector in zip(XNN, yNN):
        backpropagate(network, input_vector, target_vector)

In [102]:
def predict(input):
        return feed_forward(network, input)[-1]

## Season 7 Predictions

In [103]:
X7NN = XtestNN.values.tolist()

In [104]:
predictedY = []
for i, input in enumerate(X7NN):
    outputs = predict(input)
    predictedY.append(outputs)
#for q in predictedY:
#    print([round(a,2) for a in q])

In [105]:
predictedPlace = [a.index(max(a)) for a in predictedY]
predictedPlace = rankdata(predictedPlace,method='min')
season7NN = season7
season7NN['Predicted'] = predictedPlace
season7NN.sort_values('Actual')

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,2
9,Pearl,3,3
5,Kennedy Davenport,4,4
4,Katya,5,4
12,Trixie Mattel,6,13
7,Miss Fame,7,6
1,Jaidynn Diore Fierce,8,7
6,Max,9,8
3,Kandy Ho,10,9


In [106]:
kendalltau(season7NN.Actual,season7NN.Predicted)

KendalltauResult(correlation=0.79127622735586678, pvalue=8.0815781702633211e-05)

## Season 8 Predictions

In [107]:
season8XNN['Wins'] = scale(season8XNN['Wins'])
season8XNN['Highs'] = scale(season8XNN['Highs'])
season8XNN['Lows'] = scale(season8XNN['Lows'])
season8XNN['Lipsyncs'] = scale(season8XNN['Lipsyncs'])
X8NN = season8XNN.values.tolist()
predictedY = []
for i, input in enumerate(X8NN):
    outputs = predict(input)
    predictedY.append(outputs)
for q in predictedY:
    print([round(a,2) for a in q])

[0.0, 0.0, 0.01, 0.0, 0.01, 0.0, 0.0, 0.0, 0.94, 0.27, 0.0, 0.12, 0.0, 0.0]
[0.01, 0.97, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04, 0.03, 0.0, 0.0, 0.01]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08, 0.07, 0.29, 0.2, 0.21]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08, 0.07, 0.29, 0.2, 0.21]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.58, 0.0, 0.86, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.83, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.26, 0.04, 0.04]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.86, 0.23, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08, 0.07, 0.29, 0.19, 0.2]
[0.05, 0.0, 0.0, 0.0, 0.0, 0.14, 0.0, 0.97, 0.0, 0.0, 0.61, 0.0, 0.0, 0.0]
[0.13, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 0.22, 0.14, 0.0, 0.0, 0.0, 0.0]


In [108]:
[a.index(max(a)) for a in predictedY]

[8, 1, 0, 11, 11, 7, 7, 11, 8, 11, 7, 8]

In [109]:
predictedPlace = [a.index(max(a)) for a in predictedY]
predictedPlace = rankdata(predictedPlace,method='min')
season8NN = season8
season8NN['Predicted'] = predictedPlace
season8NN.sort_values('Predicted')

,Name,Actual,Predicted
2,Chi Chi DeVayne,0,1
1,Bob the Drag Queen,0,2
5,Derrick Barry,0,3
6,Kim Chi,0,3
10,Robbie Turner,0,3
0,Acid Betty,8,6
8,Naomi Smalls,0,6
11,Thorgy Thor,0,6
3,Cynthia Lee Fontaine,10,9
4,Dax ExclamationPoint,11,9


# Evolutionary Neural Network

Evolutionary neural networks allow neural networks to evolve over time, gaining complexity and (hopefully) precision through a combination of random mutations and reproduction of "successful" genomes. Here I use neat-python to generate neural networks through a process called NeuroEvolution of Augmenting Topologies. 

In [110]:
from neat import  nn, population, statistics, visualize

In [111]:
import math
def eval_fitness(genomes):
    for g in genomes:
        net = nn.create_feed_forward_phenotype(g)

        sum_square_error = 0.0
        predictions = []
        for inputs, expected in zip(XNN, yNN):
            # Serial activation propagates the inputs through the entire network.
            output = net.serial_activate(inputs)
            #sum_square_error += (output[0] - expected) ** 2
            predictions.append(output)
        predictedPlace = [a.index(max(a)) for a in predictions]
        actualPlace = [a.index(max(a)) for a in yNN]
        t = kendalltau(predictedPlace,actualPlace)[0]
        if math.isnan(t):
            t = 0
        g.fitness = t

In [112]:
a = [1,2,3,4,5]
b = [1,1,1,1,1]
kendalltau(a,b)[1]

nan

In [113]:
pop = population.Population('dragconfig')

In [114]:
pop.run(eval_fitness,300)


 ****** Running generation 0 ****** 

Population's average fitness: 0.02565 stdev: 0.21981
Best fitness: 0.44208 - size: (5, 19) - species 1 - id 3
Species length: 1 totaling 150 individuals
Species ID       : [1]
Species size     : [150]
Species age      : [0]
Species no improv: {1: 0}
Average adjusted fitness: 0.000
Spawn amounts: [150]
Species fitness  : [0.00017097126592920369]
Generation time: 0.634 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.16627 stdev: 0.33998
Best fitness: 0.46648 - size: (5, 19) - species 1 - id 160
Species length: 1 totaling 150 individuals
Species ID       : [1]
Species size     : [150]
Species age      : [1]
Species no improv: {1: 0}
Average adjusted fitness: 0.001
Spawn amounts: [150]
Species fitness  : [0.0011084951573194029]
Generation time: 0.651 sec

 ****** Running generation 2 ****** 

Population's average fitness: 0.21525 stdev: 0.35238
Best fitness: 0.47342 - size: (5, 21) - species 1 - id 357
Species length: 2 tota

In [115]:
winner = pop.statistics.best_genome()
print('\nBest genome:\n{!s}'.format(winner))


Best genome:
Nodes:
	NodeGene(id=0, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=1, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=2, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=3, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=4, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=5, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=6, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=7, type=INPUT, bias=0.0, response=4.924273, activation=sigmoid)
	NodeGene(id=8, type=OUTPUT, bias=-4.6415460672312765, response=5.073903254369697, activation=sigmoid)
	NodeGene(id=9, type=OUTPUT, bias=4.989974284302545, response=4.832394424728058, activation=sigmoid)
	NodeGene(id=10, type=OUTPUT, bias=-2.6587969603834334, response=4.706048159973138, activation=sigmoid)
	NodeGene(id=11, type=OUTPUT, bias=-2.4340089774183906,

In [116]:
winner_net = nn.create_feed_forward_phenotype(winner)

predicted = []
for inputs in XNN:
    output = winner_net.serial_activate(inputs)
    predicted.append(output)
predictedPlace = [a.index(max(a)) for a in predicted]
actualPlace = [a.index(max(a)) for a in yNN]
kendalltau(predictedPlace,actualPlace)[0]

0.73850689293171834

## Season 7 Predictions

In [117]:
predicted = []
for inputs in X7NN:
    output = winner_net.serial_activate(inputs)
    predicted.append(output)
predictedPlace = [a.index(max(a)) for a in predicted]
predictedPlace = rankdata(predictedPlace,method='min')
season7NN = season7
season7NN['Predicted'] = predictedPlace
season7NN.sort_values('Actual')

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,1
9,Pearl,3,6
5,Kennedy Davenport,4,3
4,Katya,5,3
12,Trixie Mattel,6,9
7,Miss Fame,7,6
1,Jaidynn Diore Fierce,8,6
6,Max,9,12
3,Kandy Ho,10,13


In [118]:
kendalltau(season7NN.Actual,season7NN.Predicted)

KendalltauResult(correlation=0.57188626358959704, pvalue=0.0043854904369570631)